# Preprocessing

In [149]:
import pandas as pd
import json
import numpy as np

# Function to load JSON file into Pandas DataFrame
def load_json(filename):
    """Loads a JSON file into a pandas DataFrame"""
    data = []
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Load Yelp dataset
business_df = load_json(r"C:\BIANCONERI\Master's AI SJSU\5- Advanced Data Mining\Final Project\Yelp-JSON\Yelp JSON\yelp_dataset\yelp_academic_dataset_business.json")
review_df = load_json(r"C:\BIANCONERI\Master's AI SJSU\5- Advanced Data Mining\Final Project\Yelp-JSON\Yelp JSON\yelp_dataset\yelp_academic_dataset_review.json")
print(f"business_df {business_df.shape}")
print(f"review_df {review_df.shape}")

# Filter only OPEN businesses in Los Angeles County with 'categories' column exists and is not null
business_df = business_df[
    (business_df['state'] == 'CA') & 
    (business_df['is_open'] == 1) & 
    (business_df['categories'].notna()) & 
    (business_df['latitude'] >= 33.7037) & 
    (business_df['latitude'] <= 34.8233)
]

print(f"business_df {business_df.shape}")
print(business_df.head())

business_df (150346, 14)
review_df (6990280, 9)
business_df (4042, 14)
                business_id                             name  \
26   noByYNtDLQAra9ccqxdfDw                              H&M   
85   IDtLPgUrqorrpqSLdfMhZQ             Helena Avenue Bakery   
91   nUqrF-h9S7myCcvNDecOvw             Iron Horse Auto Body   
120  bYjnX_J1bHZob10DoSFkqQ      Tinkle Belle Diaper Service   
141  SZU9c8V2GuREDN5KgyHFJw  Santa Barbara Shellfish Company   

                   address           city state postal_code   latitude  \
26        827-833 State St  Santa Barbara    CA       93101  34.420209   
85   131 Anacapa St, Ste C  Santa Barbara    CA       93101  34.414445   
91          825 Cacique St  Santa Barbara    CA       93103  34.419620   
120                         Santa Barbara    CA       93101  34.420334   
141      230 Stearns Wharf  Santa Barbara    CA       93101  34.408715   

      longitude  stars  review_count  is_open  \
26  -119.700460    3.0            24        1   
8

In [150]:
# Extract and print all unique categories from business_df
unique_categories = set()
business_df['categories'].apply(lambda x: unique_categories.update(x.split(', ')))
print(f"Number of unique categories: {len(unique_categories)}")
print(unique_categories)

Number of unique categories: 945
{'Body Shops', 'Registration Services', 'Party Supplies', 'Web Design', 'Furniture Stores', 'Department Stores', 'Cocktail Bars', 'Structural Engineers', 'Urologists', 'Advertising', 'Water Delivery', 'Vintage & Consignment', 'Propane', 'Halfway Houses', 'Landscaping', 'Screen Printing', 'Landscape Architects', 'Career Counseling', 'Boudoir Photography', 'Chicken Shop', 'Grocery', 'Shredding Services', 'Pumpkin Patches', 'Office Cleaning', 'Appliances', 'Venues & Event Spaces', 'Orthopedists', 'Souvenir Shops', 'Hobby Shops', 'Karate', 'Education', 'Employment Law', 'Dry Cleaning & Laundry', 'Lounges', 'Escape Games', 'Vietnamese', 'Furniture Assembly', 'Knife Sharpening', 'Car Stereo Installation', 'Gun/Rifle Ranges', 'Prosthodontists', 'Medical Spas', 'Tree Services', 'Wineries', 'Pop-up Shops', 'Swimwear', 'Car Window Tinting', 'New Mexican Cuisine', 'Cheese Shops', 'Kickboxing', 'Powder Coating', 'Food', 'Gemstones & Minerals', 'Tiki Bars', 'General

In [151]:
# Convert categories to lowercase for consistency
business_df['categories'] = business_df['categories'].apply(lambda x: [category.lower() for category in x.split(', ')])

In [152]:
# Define restaurant-related keywords
# restaurant_keywords = ["bars","donuts","barbeque", "sandwiches","wineries","fish & chips","vegetarian", "beer", "food", "dessert", "gelato", "restaurants", "wine", "tacos", "tea", "acai bowls", "whiskey", "juice bars & smoothies", "poke", "spirits", "cocktail", "salad", "coffee", "bakeries", "breweries", "pizza", "burgers", "soup", "bagels", "ice cream & frozen yogurt", "ramen", "chicken wings", "food trucks", "cafes", "seafood", "vegan", "diners", "noodles"]
restaurant_keywords = ["barbeque","sandwiches","fish & chips","vegetarian","restaurants","tacos","poke","salad", "pizza", "burgers", "soup", "ramen", "chicken wings", "seafood", "vegan", "diners", "noodles","comfort food", "asian fusion", "tex-mex", "dim sum", "smokehouse"]

# Filter RESTAURANTS ONLY (businesses with restaurant-related keywords)
restaurants_df = business_df[business_df['categories'].apply(lambda x: any(keyword in x for keyword in restaurant_keywords))]

# Select relevant columns // we may get rid of address related columns if we won't use them
restaurants_df = restaurants_df[['business_id', 'name', 'address', 'city', 'postal_code', 'latitude', 'longitude', 'categories', 'stars', 'review_count']]

print(f"restaurants_df {restaurants_df.shape}")
print(restaurants_df.head())

# Drop rows with missing values // there is none
#df_clean = restaurants_df.dropna()
#print(f"df_clean {df_clean.shape}")

restaurants_df (668, 10)
                business_id                             name  \
85   IDtLPgUrqorrpqSLdfMhZQ             Helena Avenue Bakery   
141  SZU9c8V2GuREDN5KgyHFJw  Santa Barbara Shellfish Company   
431  ifjluUv4VASwmFqEp8cWlQ                    Marty's Pizza   
470  VeFfrEZ4iWaecrQg6Eq4cg                         Cal Taco   
555  bdfZdB2MTXlT6-RBjSIpQg                       Pho Bistro   

                       address           city postal_code   latitude  \
85       131 Anacapa St, Ste C  Santa Barbara       93101  34.414445   
141          230 Stearns Wharf  Santa Barbara       93101  34.408715   
431         2733 De La Vina St  Santa Barbara       93105  34.436236   
470  7320 Hollister Ave, Ste 1         Goleta       93117  34.430542   
555  903 Embarcadero Del Norte     Isla Vista       93117  34.412934   

      longitude                                         categories  stars  \
85  -119.690672  [food, restaurants, salad, coffee & tea, break...    4.0   
141

In [153]:
# Select relevant columns from review_df //dropped review_id, useful, funny, cool
review_df = review_df[['user_id','business_id','stars','date','text']]
print(review_df.head())

# Merge reviews with restaurant using business_id as a key
merged_df = review_df.merge(restaurants_df[['business_id', 'name', 'categories']], on='business_id')
print(f"merged_df {merged_df.shape}")
print(merged_df.head())

                  user_id             business_id  stars                 date  \
0  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw    3.0  2018-07-07 22:09:11   
1  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ    5.0  2012-01-03 15:28:18   
2  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A    3.0  2014-02-05 20:30:30   
3  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA    5.0  2015-01-04 00:01:03   
4  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ    4.0  2017-01-14 20:54:15   

                                                text  
0  If you decide to eat here, just be aware it is...  
1  I've taken a lot of spin classes over the year...  
2  Family diner. Had the buffet. Eclectic assortm...  
3  Wow!  Yummy, different,  delicious.   Our favo...  
4  Cute interior and owner (?) gave us tour of up...  
merged_df (167713, 7)
                  user_id             business_id  stars                 date  \
0  59MxRhNVhU9MYndMkz0wtw  gebiRewfieSdtt17PTW6Zg    3.0  2016-07-25 07:31:06   

In [154]:
# Fix category column if it's a list -> string
merged_df['categories'] = merged_df['categories'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
print(merged_df.head())

                  user_id             business_id  stars                 date  \
0  59MxRhNVhU9MYndMkz0wtw  gebiRewfieSdtt17PTW6Zg    3.0  2016-07-25 07:31:06   
1  OhECKhQEexFypOMY6kypRw  vC2qm1y3Au5czBtbhc-DNw    4.0  2013-09-04 03:48:20   
2  4hBhtCSgoxkrFgHa4YAD-w  bbEXAEFr4RYHLlZ-HFssTA    5.0  2017-01-02 03:17:34   
3  bFPdtzu11Oi0f92EAcjqmg  IDtLPgUrqorrpqSLdfMhZQ    5.0  2016-10-13 22:50:47   
4  JYYYKt6TdVA4ng9lLcXt_g  SZU9c8V2GuREDN5KgyHFJw    5.0  2016-05-31 02:14:54   

                                                text  \
0  Had a party of 6 here for hibachi. Our waitres...   
1  Yes, this is the only sushi place in town. How...   
2  Great burgers,fries and salad!  Burgers have a...   
3  What a great addition to the Funk Zone!  Grab ...   
4  We were a bit weary about trying the Shellfish...   

                              name  \
0  Hibachi Steak House & Sushi Bar   
1                       Sushi Teri   
2  The Original Habit Burger Grill   
3             Helena Ave

In [155]:
### PLACEHOLDER for the Train/Test Split for merged_df ### 
# Note: The test set contains the reviews after the cutoff that were made by users with at least one review during the training period.

from sklearn.model_selection import train_test_split

# Split the data into train nd test sets (80,20 split) for now
#train_data, test_data = train_test_split(merged_df, test_size=0.2, random_state=42) # to be replaced by the actual tain_data & test_data

merged_df['yyyy-mm'] = merged_df['date'].str[0:7]
merged_df['monthnum'] = merged_df['yyyy-mm'].str.replace("-","")
monthlies = pd.DataFrame()
monthlies['counts'] = merged_df.groupby('yyyy-mm').size()
monthlies['cumulative'] = monthlies['counts'].cumsum()

monthlies.head()
monthlies.tail()
total = monthlies['cumulative'].iloc[-1]
splitpt = total * 0.8
print(splitpt)

monthlies['splitabs'] = abs(monthlies['cumulative']-splitpt)
splitmth = monthlies['splitabs'].argmin()
splitstr = monthlies.iloc[splitmth].name.replace("-","")
splitstr

train_data = pd.DataFrame
test_data = pd.DataFrame
train_data = merged_df[merged_df['monthnum'] < splitstr]
test_data = merged_df[merged_df['monthnum'] > splitstr]

134170.4


# Collaborative Filtering

In [101]:
from sklearn.metrics.pairwise import cosine_similarity

# User-Restaurant interaction matrix for Collaborative Filtering using 'stars'
user_restaurant_matrix = train_data.pivot_table(index='user_id', columns='business_id', values='stars', fill_value=0)
print(user_restaurant_matrix.head())

# Calculate the total number of elements in the matrix
total_elements = user_restaurant_matrix.size
# Calculate the number of non-zero elements
non_zero_elements = (user_restaurant_matrix != 0).sum().sum()
# Calculate density
density = non_zero_elements / total_elements
# Calculate sparsity
sparsity = 1 - density
print(f"Total Elements: {total_elements}")
print(f"Non-Zero Elements: {non_zero_elements}")
print(f"Density: {density:.4f}")
print(f"Sparsity: {sparsity:.4f}")

# Compute user similarity matrix
user_similarity = cosine_similarity(user_restaurant_matrix)
print(user_similarity)

business_id             -3AooxIkg38UyUdlz5oXdw  -6jvfSJGprbfBD2QrS9zQw  \
user_id                                                                  
---zemaUC8WeJeWKqS6p9Q                     0.0                     0.0   
--17Db1K-KujRuN7hY9Z0Q                     0.0                     0.0   
--2F5G5LKt3h2cAXJbZptg                     0.0                     0.0   
--2vR0DIsmQ6WfcSzKWigw                     0.0                     0.0   
--50YzjtBsdxOGVqTkvaKA                     0.0                     0.0   

business_id             -9r8nAzWyRSLxBWt8uQOdA  -ALqLSTzkGDMscHdxA1NgA  \
user_id                                                                  
---zemaUC8WeJeWKqS6p9Q                     0.0                     0.0   
--17Db1K-KujRuN7hY9Z0Q                     0.0                     0.0   
--2F5G5LKt3h2cAXJbZptg                     0.0                     0.0   
--2vR0DIsmQ6WfcSzKWigw                     0.0                     0.0   
--50YzjtBsdxOGVqTkvaKA               

MemoryError: Unable to allocate 54.4 GiB for an array with shape (85483, 85483) and data type float64

In [156]:
# Filter users with at least 5 ratings to save on memory
user_counts = train_data['user_id'].value_counts()
filtered_df = train_data[train_data['user_id'].isin(user_counts[user_counts >= 5].index)]

# Filter restaurants with at least 5 ratings to save on memory
item_counts = filtered_df['business_id'].value_counts()
filtered_df = filtered_df[filtered_df['business_id'].isin(item_counts[item_counts >= 5].index)]

# Recompute User-Restaurant interaction matrix using filtered_df
user_restaurant_matrix = filtered_df.pivot_table(index='user_id', columns='business_id', values='stars', fill_value=0)
print(user_restaurant_matrix.head())

# Calculate the total number of elements in the matrix
total_elements = user_restaurant_matrix.size
# Calculate the number of non-zero elements
non_zero_elements = (user_restaurant_matrix != 0).sum().sum()
# Calculate density
density = non_zero_elements / total_elements
# Calculate sparsity
sparsity = 1 - density
print(f"Total Elements: {total_elements}")
print(f"Non-Zero Elements: {non_zero_elements}")
print(f"Density: {density:.4f}")
print(f"Sparsity: {sparsity:.4f}")

# Recompute user similarity matrix
user_similarity = cosine_similarity(user_restaurant_matrix)
print(user_similarity)

business_id             -3AooxIkg38UyUdlz5oXdw  -9r8nAzWyRSLxBWt8uQOdA  \
user_id                                                                  
-0EcgtUXe1rzrkmdih_tYg                     0.0                     0.0   
-1-ECBsGpG4Iw5s-ecnfqw                     0.0                     0.0   
-14MA777BbjUQLw0zndvfA                     0.0                     0.0   
-1WbN1Qd-opw8u3uEqs2Kg                     0.0                     0.0   
-2ynqM2Z6pqzdUH6YXz7iQ                     0.0                     0.0   

business_id             -ALqLSTzkGDMscHdxA1NgA  -FM4CxOg4XXmX_Ebky_SiQ  \
user_id                                                                  
-0EcgtUXe1rzrkmdih_tYg                     0.0                     0.0   
-1-ECBsGpG4Iw5s-ecnfqw                     0.0                     5.0   
-14MA777BbjUQLw0zndvfA                     0.0                     0.0   
-1WbN1Qd-opw8u3uEqs2Kg                     0.0                     0.0   
-2ynqM2Z6pqzdUH6YXz7iQ               

In [166]:
from collections import defaultdict

def get_cf_recommendations(user_id, user_similarity, user_restaurant_matrix, merged_df, top_n=5):
    """
    Generate restaurant recommendations for a user using collaborative filtering with weighted scores.
    
    Args:
        user_id (str): The ID of the user for whom recommendations are generated.
        user_similarity (numpy array): User similarity matrix.
        user_restaurant_matrix (DataFrame): User-restaurant interaction matrix.
        merged_df (DataFrame): The original DataFrame containing user_id, business_id, stars, text, name, categories.
        top_n (int): Number of recommendations to return.
    
    Returns:
        list: List of recommended restaurant IDs.
    """
    # Get the index of the user in the user-item matrix
    user_index = user_restaurant_matrix.index.get_loc(user_id)
    
    # Get the similarity scores for the user
    user_sim_scores = user_similarity[user_index]
    
    # Get the indices of the most similar users (excluding the user themselves)
    similar_users_indices = user_sim_scores.argsort()[::-1][1:top_n + 1]
    
    # Initialize a dictionary to accumulate weighted scores for each restaurant
    restaurant_weighted_scores = defaultdict(float)
    
    # Iterate over similar users
    for sim_user_index in similar_users_indices:
        sim_user_id = user_restaurant_matrix.index[sim_user_index]
        sim_user_similarity = user_sim_scores[sim_user_index]  # Similarity score of the similar user
        
        # Get restaurants rated highly by the similar user
        sim_user_ratings = user_restaurant_matrix.loc[sim_user_id]
        highly_rated_restaurants = sim_user_ratings[sim_user_ratings >= 4]  # Restaurants rated 4 or 5 stars ONLY
        
        # Accumulate weighted scores for each restaurant
        for restaurant_id, rating in highly_rated_restaurants.items():
            restaurant_weighted_scores[restaurant_id] += rating * sim_user_similarity
    
    # Exclude restaurants already rated by the user
    user_rated_restaurants = user_restaurant_matrix.loc[user_id][user_restaurant_matrix.loc[user_id] > 0].index
    for restaurant_id in user_rated_restaurants:
        if restaurant_id in restaurant_weighted_scores:
            del restaurant_weighted_scores[restaurant_id]
    
    # Sort restaurants by weighted score in descending order
    sorted_restaurants = sorted(restaurant_weighted_scores.keys(), key=lambda x: restaurant_weighted_scores[x], reverse=True)
    
    # Get the top n restaurants
    top_restaurants = sorted_restaurants[:top_n]
    
    # Get the names & categories of the recommended restaurants
    recommended_restaurant_names = merged_df[merged_df['business_id'].isin(top_restaurants)]['name'].unique()
    print("CF Recommended Restaurant Names:", list(recommended_restaurant_names)[:top_n])
    print()
    recommended_restaurant_categories = merged_df[merged_df['business_id'].isin(top_restaurants)]['categories'].unique()
    print("CF Recommended Restaurant Categories:", list(recommended_restaurant_categories)[:top_n])
    print()
    
    return top_restaurants

In [167]:
# Test: Get CF recommendations for a specific user
user_id = '-0EcgtUXe1rzrkmdih_tYg'
cf_recommendations = get_cf_recommendations(user_id, user_similarity, user_restaurant_matrix, merged_df, top_n=5)
print("Recommended Restaurants:", cf_recommendations)

CF Recommended Restaurant Names: ['Intermezzo By Wine Cask', "Norton's Pastrami & Deli", 'The Habit Burger Grill', 'Los Agaves', 'Freebirds']

CF Recommended Restaurant Categories: ['tapas bars, nightlife, salad, restaurants, cocktail bars, bars, wine bars, cafes, pizza, american (new)', 'gluten-free, american (new), restaurants, sandwiches, salad, steakhouses, delis, american (traditional)', 'burgers, restaurants', 'mexican, restaurants', 'salad, restaurants, tacos, mexican, breakfast & brunch']

Recommended Restaurants: ['vhWlxyLEWAbNOMtQ42yzFg', 'PPcz-0UtIJ8O7xDT551y2g', '9u79dIXLQzmt9rnOZEnBaQ', 'Rl42JbSMsmNW3LRjsTMYAg', 'yPSejq3_erxo9zdVYTBnZA']


# Content-based Filtering

In [180]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_cb_recommendations(cf_recommendations, user_id, merged_df, top_n=5):
    """
    Refine CF recommendations using content-based filtering with categories and text reviews.
    
    Args:
        cf_recommendations (list): List of restaurant IDs recommended by CF.
        user_id (str): The ID of the user for whom recommendations are generated.
        merged_df (DataFrame): The original DataFrame containing restaurant categories and reviews.
        top_n (int): Number of recommendations to return.
    
    Returns:
        list: List of refined restaurant IDs.
    """
    # Get the user's preferred categories and reviews (from their rated restaurants)
    user_rated_restaurants = merged_df[merged_df['user_id'] == user_id]['business_id']
    user_data = merged_df[merged_df['business_id'].isin(user_rated_restaurants)]
    
    # Combine categories and reviews for the user
    user_categories = user_data['categories'].str.cat(sep=', ')
    user_reviews = user_data['text'].str.cat(sep=' ')
    user_combined = user_categories + " " + user_reviews  # Combine categories and reviews
    
    # Get the categories and reviews of the CF-recommended restaurants
    #recommended_data = merged_df[merged_df['business_id'].isin(cf_recommendations)][['business_id', 'categories', 'text']]
    recommended_data = (
        merged_df[merged_df['business_id'].isin(cf_recommendations)]
        .groupby('business_id')
        .agg({'categories': 'first', 'text': ' '.join})  # Combine all reviews into a single string
        .reset_index()
    )
    
    # Combine categories and reviews for the recommended restaurants
    recommended_data['combined'] = recommended_data['categories'] + " " + recommended_data['text']
    
    # Compute TF-IDF vectors for user combined data and recommended restaurant combined data
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform([user_combined] + recommended_data['combined'].tolist())
    
    # Compute cosine similarity between user combined data and recommended restaurant combined data
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    
    # Add similarity scores to the recommended_data DataFrame
    recommended_data['similarity'] = similarity_scores
    
    # Sort recommended restaurants by similarity score in descending order
    sorted_recommendations = recommended_data.sort_values(by='similarity', ascending=False)['business_id'].tolist()
    
    # Return the top n refined recommendations
    return sorted_recommendations[:top_n]

# Hybrid Recommender

In [178]:
def sequential_recommender(user_id, user_similarity, user_restaurant_matrix, merged_df, top_n=5):
    """
    Generate recommendations using a sequential CF → CB approach.
    
    Args:
        user_id (str): The ID of the user for whom recommendations are generated.
        user_similarity (numpy array): User similarity matrix.
        user_restaurant_matrix (DataFrame): User-restaurant interaction matrix.
        merged_df (DataFrame): The original DataFrame containing restaurant names, categories, and reviews.
        top_n (int): Number of recommendations to return.
    
    Returns:
        list: List of final recommended restaurant IDs.
    """
    # Step 1: Collaborative Filtering (CF)
    cf_recommendations = get_cf_recommendations(user_id, user_similarity, user_restaurant_matrix, merged_df, top_n * 2)  # Get more recommendations for refinement
    
    # Step 2: Content-Based Filtering (CB)
    final_recommendations = get_cb_recommendations(cf_recommendations, user_id, merged_df, top_n)
    
    return final_recommendations

In [181]:
# Get final recommendations for a target user
user_id = '-0EcgtUXe1rzrkmdih_tYg'
final_recommendations = sequential_recommender(user_id, user_similarity, user_restaurant_matrix, merged_df, top_n=5)

# Get the names & categories of the final recommendations
final_recommendation_names = merged_df[merged_df['business_id'].isin(final_recommendations)]['name'].unique()
final_recommendation_categories = merged_df[merged_df['business_id'].isin(final_recommendations)]['categories'].unique()

print("Final Recommended Restaurant Names:", list(final_recommendation_names))
print()
print("Final Recommended Restaurant Categories:", list(final_recommendation_categories))

CF Recommended Restaurant Names: ['Intermezzo By Wine Cask', 'Brasil Arts Cafe', "Renaud's Patisserie & Bistro", "Norton's Pastrami & Deli", 'The Habit Burger Grill', '101 Deli', 'Los Agaves', 'Freebirds', "Andersen's Danish Bakery & Restaurant"]

CF Recommended Restaurant Categories: ['tapas bars, nightlife, salad, restaurants, cocktail bars, bars, wine bars, cafes, pizza, american (new)', 'juice bars & smoothies, breakfast & brunch, brazilian, restaurants, food, salad, acai bowls', 'coffee & tea, breakfast & brunch, restaurants, french, salad, sandwiches, desserts, bakeries, food', 'gluten-free, american (new), restaurants, sandwiches, salad, steakhouses, delis, american (traditional)', 'burgers, restaurants', 'delis, restaurants', 'mexican, restaurants', 'salad, restaurants, tacos, mexican, breakfast & brunch', 'modern european, nightlife, bars, restaurants, bakeries, wine bars, food, scandinavian, breakfast & brunch', 'french, cafes, restaurants, breakfast & brunch, food, sandwiche